# 1. Data Compress Method with Discrete Morse Theory

In [67]:
import numpy as np
import json

In [182]:
def compressIt(pnlist,nodeTypeList, upLimit):
    """Testing. the main function to compress the polyhedra complex by coordinates. pnlist is from the parser 
    read from DSGRN and nodeTypeList is a list of the PN.json file names"""
    
    
    candidates = set(range(len(pnlist[0])))
    buffers = set()
    singles = set()
    currentPNList = pnlist.copy()
    
    count = 0
    while (candidates != (buffers|singles)) and (count < upLimit) :
        count+=1
        print('-'*40+' round starts ' + '-'*40)
        print("pre states")
        print("candidates: ",candidates)
        print("buffers: ",buffers)
        
        coordinate = selectCoordinate(currentPNList, candidates, buffers,singles)
        print("coordinate: ", coordinate )
        currentAtomicPNList = list(set([x[coordinate] for x in currentPNList]))
        
        fiberDict = generateFiberDictionary(currentPNList,coordinate)
        
        
        nodetype = nodeTypeList[coordinate]
        topCellAdjacencyInfoByCoordinate = loadAtomicChainComplex(nodetype)
        updatedPNList = compressSelectedCoordinate(nodetype,currentPNList,coordinate,fiberDict,topCellAdjacencyInfoByCoordinate)
        
        if len(updatedPNList) == len(currentPNList):
            buffers.add(coordinate)
        else:
            buffers = set([coordinate])
        
        
            
        print("curr:{}".format(len(currentPNList)),"updated:{}".format(len(updatedPNList)))
        
        currentPNList = updatedPNList
        isSingleNodeAndUpdate(currentPNList,coordinate,singles)
        
        
        print("post states")
        print("candidates: ",candidates)
        print("buffers: ",buffers)
        print("singles: ", singles)
        print('-'*40+' round ends ' + '-'*40)
        
    return currentPNList
        
        
        
        
def generateFiberDictionary(currentPNList,coordinate):
    fiberDict = dict()
    for pn in currentPNList:
        key = pn[coordinate]
        if key not in fiberDict:
            fiberDict[key] = set()
        
        coordRemovedPN = tuple(list(pn[:coordinate]) + list(pn[coordinate+1:]))
        fiberDict[key].add(coordRemovedPN)
        
    return fiberDict


def selectCoordinate(currentPNList,candidates,buffers,singles):
    """Done. select the one with smallest base"""
    
    diff = candidates - buffers - singles
    l = float("Inf")
    ret = -1
    for coord in diff:
        baseLength = len(set([x[coord] for x in currentPNList]))
        print("coord:",coord, "base length:", baseLength)
        if baseLength < l:
            l = baseLength
            ret = coord
    
    return ret



def compressSelectedCoordinate(nodetype,currentPNList, coordinate,fiberDict,topCellAdjacencyInfoByCoordinate):
    """Done"""
    
    #print('currentPNList pre',currentPNList)
    atomicPNList = list(set([x[coordinate] for x in currentPNList]))
    orderedAtomicPNList = selectAPNByFiberLength(atomicPNList,fiberDict)
    #print("fiberDict:",fiberDict)
    #print(orderedAtomicPNList)
    for apn in orderedAtomicPNList:
        if isCompressible(nodetype,apn,fiberDict,topCellAdjacencyInfoByCoordinate,atomicPNList):
            print("is compressed:", apn)
            ret = []
            for i in range(len(currentPNList)):
                currPN = currentPNList[i]
                if currPN[coordinate]!= apn:
                    ret.append(currPN)
            currentPNList = ret
            atomicPNList.remove(apn)
    
    #print('currentPNListpost',currentPNList)
    return tuple(currentPNList)


def selectAPNByFiberLength(atomicPNList,fiberDict):
    """Done. We can select atomic parameter with increasing order of the fiber length."""
    ret = []
    lengthList = [len(fiberDict[x]) for x in atomicPNList]
    orders = np.argsort(lengthList)
    for i in range(len(orders)):
        order = orders[i]
        ret.append(atomicPNList[order])
    return ret

def isCompressible(nodetype, apn, fiberDict,topCellAdjacencyInfoByCoordinate, atomicPNList):
    """Half Done. Wait for discreteMorse interface. Need the preAtt discrete morse method"""
    
    # if fiber is minimal in its neighborhood, continue, else return False
    adjacentAPNInfo = getAdjacentTopCell(apn,topCellAdjacencyInfoByCoordinate)
    #print(apn," has adjacent top cells", adjacentTopCells)
    adjacentAPN = []
    for currAPN in atomicPNList:
        if currAPN in adjacentAPNInfo:
            adjacentAPN.append(currAPN)
            
    intersectionDict = getAPNInteractionDict(apn, adjacentAPN)
    removeRedundantDict = removeRedundantIntersection(intersectionDict)
    clearNeighborAPN = list(removeRedundantDict.keys())
    
    apnFiber = fiberDict[apn]
    
    print("start checking fiber")
    #print("the neightAPN need to check: ",clearNeighborAPN)
    print("not redundance neighboring apn: ", clearNeighborAPN)
    for currAPN in clearNeighborAPN:
        if currAPN != apn:
            currFiber = fiberDict[currAPN]
            if apnFiber.issubset(currFiber):
                continue
            else:
                print("the intersection failed pair: ",apn,currAPN)
                return False
            
    
    # is the ace an empty set, if it is return True, else return False
    print("start testing the ace is empty or not")
    king, queen, ace = discreteMorseWithPreAtt(nodetype,apn,atomicPNList)
    
    if len(ace) == 0:
        return True
    else:
        return False
    
def getAPNIntersection(firstAPN, secondAPN):
    ret = set()
    for i in range(len(firstAPN)):
        pre = firstAPN[i]
        for j in range(i+1,len(firstAPN)):
            post = firstAPN[j]
            if secondAPN.index(pre) > secondAPN.index(post):
                ret.add(tuple(sorted([pre,post])))   
                
    return ret

def getAPNInteractionDict(apn, adjacentAPN):
    ret = dict()
    for adjAPN in adjacentAPN:
        intersection = getAPNIntersection(apn,adjAPN)
        ret[adjAPN] = intersection
        
    return ret

def removeRedundantIntersection(apnIntersectionDict):
    """remove lower dimension intersection if it is contained in some higher dimension intersection"""
    ret = dict()
    apnList  = list(apnIntersectionDict.keys())
    redundantAPNSet = set()
    for i in range(len(apnList)):
        for j in range(len(apnList)):
            
            if i == j:
                continue
    
            intersectionJ = apnIntersectionDict[apnList[j]]
            intersectionI = apnIntersectionDict[apnList[i]]

            if intersectionI.issuperset(intersectionJ):
                redundantAPNSet.add(apnList[i])
                
    for k in apnIntersectionDict:
        if k not in redundantAPNSet:
            ret[k] = apnIntersectionDict[k]
    
    return ret
    

def isSingleNodeAndUpdate(currentPNList,coordinate,singles):
    """Done."""
    
    #print("length of base: ", len(set([x[coordinate] for x in currentPNList])))
    if len(set([x[coordinate] for x in currentPNList])) == 1:
        singles.add(coordinate)
        
    return 


def getAdjacentTopCell(apn,topCellAdjacencyInfoByCoordinate):
    """Done."""
    dim = len(topCellAdjacencyInfoByCoordinate[apn])-1
    ret = set()
    for currAPN in topCellAdjacencyInfoByCoordinate:
        if currAPN != apn:
            topCells = set(list(topCellAdjacencyInfoByCoordinate[apn][dim])[0])
            currTopCells = set(list(topCellAdjacencyInfoByCoordinate[currAPN][dim])[0])
            #print("apn top cell",apn,topCells)
            #print("neighbor top cell",currAPN, currTopCells)
            if len(topCells & currTopCells) >0:
                ret.add(currAPN)
    return ret

def getTopCellAdajcencyForAtomicPN(nodetype):
    """Done."""
    ret = loadAtomicChainComplex(nodeType)
    return ret

In [186]:
updatedPNList = compressIt(pnlist, nodeTypeList,4)

---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2, 3, 4}
buffers:  set()
coord: 0 base length: 1
coord: 1 base length: 6
coord: 2 base length: 4
coord: 3 base length: 4
coord: 4 base length: 172
coordinate:  0
start checking fiber
not redundance neighboring apn:  []
start testing the ace is empty or not
curr:6904 updated:6904
post states
candidates:  {0, 1, 2, 3, 4}
buffers:  {0}
singles:  {0}
---------------------------------------- round ends ----------------------------------------
---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2, 3, 4}
buffers:  {0}
coord: 1 base length: 6
coord: 2 base length: 4
coord: 3 base length: 4
coord: 4 base length: 172
coordinate:  2
start checking fiber
not redundance neighboring apn:  [(0, -2, -1, 1)]
start testing the ace is empty or not
is compressed: (-2, 0, -1, 1)
start checking fiber
not redund

In [70]:
def loadAtomicChainComplex(fpath):
    ret = dict()
    with open(fpath,'r') as file:
        atomicCC = json.load(file)
    for key in atomicCC:
        newk = key[1:-1].split(',')
        newk = tuple([int(x) for x in newk])
        v = atomicCC[key]
        newvalue = dict()
        for dim in v:
            newfaces = set([tuple(x) for x in v[dim]])
            newvalue[int(dim)] = newfaces
        ret[newk] = newvalue 
    return ret


def checkCoreduction(face, coboundary, currComplex,used):
    for cell in coboundary:        
        if len(currComplex[cell] - used) == 1:
            return True
    return False


# used is the preset attractors
# currComplex is a dict the key as cell, and values a set of its (proper) faces, its inverse dict is coboundaryMatrix 
def genMatchingWithPreAtt(currComplex,coboundaryMatrix,used):
    
    kings = []
    queens = []
    aces = []
    
    count = 0
    while currComplex:
        count +=1
        
        # find queen, king pairs
        
        flag = True
        
        # flag:  find coreducible pair in previous around
        while flag:
            tempComplex = dict()
            flag = False
            for cell in currComplex:

                tempDiff = currComplex[cell] - used

                if len(tempDiff) == 1:
                    queen = list(tempDiff)[0]
                    king = cell

                    queens.append(queen)
                    kings.append(king)

                    used.add(king)
                    used.add(queen)

                    flag = True
                # if not paired cell
                if cell not in used:
                    tempComplex[cell] = tempDiff
            
            currComplex = tempComplex
            
        #print(queens, kings, aces)
        #print(tempComplex)
        
        # check coreducible or not
        flag = False # exist coreducible or not
        while not flag:
            tempComplex = dict()
            
            for key in currComplex:
                
                # if already have coreduction in previous loop, then only restore the rest of dictionary
                if flag:
                    tempComplex[key] = currComplex[key]
                    continue
                
                
                if len(currComplex[key]) == 0:
                    aces.append(key)
                    used.add(key)

                    # if key is a top cell, just continue
                    if key not in coboundaryMatrix:
                        continue

                    # if there is a coreducible pair we break the aces collection process
                    # we are not collecting the pair here
                    if checkCoreduction(key, coboundaryMatrix[key], currComplex, used):
                        flag = True
                        
                else:
                    tempComplex[key] = currComplex[key]
            
            currComplex = tempComplex
            
            # if empty
            if not currComplex:
                break
        
    return queens, kings, aces

def isFace(f,c):
    if set(f).issubset(set(c)):
        return True
    
    return False

def getBoundaryMatrix(apnList):
    faceLattice = dict()
    for index in apnList:
        pn = apnList[index]
        d = len(pn)
        for dim in range(d-1,0,-1):
            cells = pn[dim]
            faces = pn[dim-1]
            for c in cells:
                for f in faces:
                    if isFace(f,c):
                        sf = tuple(sorted(f))
                        sc = tuple(sorted(c))
                        if sc not in faceLattice:
                            faceLattice[sc] = []
                        faceLattice[sc].append(sf)
        
        # the zero diminsion
        cells = pn[0]
        for point in cells:
            faceLattice[tuple(sorted(point))] = []
            
            
    # remove the duplicate faces
    for key in faceLattice:
        faceLattice[key] = list(set(faceLattice[key]))
            
    return faceLattice

def getCoboundaryMatrix(boundaryMatrix):
    coboundaryMatrix = dict()
    
    for cell in boundaryMatrix:
        boundary = boundaryMatrix[cell]
        for b in boundary:
            if b not in coboundaryMatrix:
                coboundaryMatrix[b] = []
            coboundaryMatrix[b].append(cell)
    return coboundaryMatrix


def discreteMorseWithPreAtt(nodetype, apn, atomicPNList):
    """Done. the cells except apn will be regarded as pre-ace set"""
    
    apnList = loadAtomicChainComplex(nodetype)
    factorAPN = dict()
    for k in atomicPNList:
        factorAPN[k] = apnList[k]
        
    boundaryMatrix = getBoundaryMatrix(factorAPN)
    coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)
    
    preAtt = set()
    for currAPN in atomicPNList:
        if currAPN!= apn:
            apnFL = factorAPN[currAPN]
            for d in apnFL:
                preAtt|= apnFL[d]
        
    tempComplex = set()
    
    apnCells = factorAPN[apn]
    for d in apnCells:
        tempComplex|= apnCells[d]
        
    tempComplex-=preAtt
    
    subComplex = dict()
    for cell in tempComplex:
        subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

    currComplex = subComplex
    used = preAtt
    king, queen, ace = genMatchingWithPreAtt(currComplex,coboundaryMatrix,used)
    
    return king, queen, ace

# 2. Discrete Morse or Direct Computation

# need the method to generate the adjacency matrix for the rest of the PNList 

# 3. Test

In [72]:
import sys
sys.path+=['/Users/lunzhang/anaconda3/lib/python3.6/site-packages']
import DSGRN
import os
import json
import time 
import re
import json
import numpy as np
from itertools import product
from itertools import permutations
import os
import sys

## toggle switch

In [73]:
nwStr = "x1:~x2 \n x2:~x1"
network = DSGRN.Network(nwStr)
DSGRN.DrawGraph(network)
parameterGraph = DSGRN.ParameterGraph(network)
parameterGraph.size()

9

In [74]:
monostable = []
size = parameterGraph.size()
for i in range(size):
    parameterSample = parameterGraph.parameter(i)
    domainGraph = DSGRN.DomainGraph(parameterSample)
    morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
    morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)
    l = morseGraph.poset().size()
    monostable.append(parameterSample)
    #print(j)
    for j in range(l):
        if morseGraph.poset().size() != 1:
            monostable = monostable[:-1]
            break  

In [75]:
# suppose there are at most 2 thresholds

def stringifyParser(l):
    ret =  []
    l = eval(l)
    #print(l)
    for i in range(len(l)):
        inputNo = l[i][1][0]  
        m=outputNo = l[i][1][1] 
        polyNo = pow(2,inputNo)
        totalLen = polyNo*outputNo

        logic = l[i][1][2]
        logic = bin(int(logic,16))[2:].zfill(totalLen)

        lower = []
        middle = []
        upper = []
        thetas = l[i][2]
        temp = []
        for j in range(0,len(logic),m):
            polyIndex = polyNo - int(j/m) - 1
            piece = logic[j:j+m]
            #print(piece)
            if len(set(piece)) == 2:
                middle.append(polyIndex)
            elif set(piece) == set('1'):
                upper.append(polyIndex)
            else:
                lower.append(polyIndex)

        if len(thetas)==1:
            temp = sorted(lower) + [-thetas[0]-1] + sorted(upper)
        else:
            
            temp = sorted(lower) + [-thetas[0]-1] + sorted(middle) + [-thetas[1]-1]+sorted(upper)
            #print(temp)

        ret.append(tuple(temp))
    return ret

In [76]:
pnlist = [stringifyParser(x.stringify()) for x in monostable]
pnlist = [tuple(x) for x in pnlist]

In [77]:
nodeTypeList =  ['celldata/x_o_1_PN.json','celldata/x_o_1_PN.json']

In [192]:
updatePNList = compressIt(pnlist, nodeTypeList,float("Inf"))

---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2}
buffers:  set()
coord: 0 base length: 3
coord: 1 base length: 3
coord: 2 base length: 3
coordinate:  0
start checking fiber
not redundance neighboring apn:  [(-1, 0, 1), (0, 1, -1)]
start testing the ace is empty or not
start checking fiber
not redundance neighboring apn:  [(0, -1, 1)]
the intersection failed pair:  (-1, 0, 1) (0, -1, 1)
start checking fiber
not redundance neighboring apn:  [(0, -1, 1)]
the intersection failed pair:  (0, 1, -1) (0, -1, 1)
curr:26 updated:26
post states
candidates:  {0, 1, 2}
buffers:  {0}
singles:  set()
---------------------------------------- round ends ----------------------------------------
---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2}
buffers:  {0}
coord: 1 base length: 3
coord: 2 base length: 3
coordinate:  1
start checking fiber
not red

In [79]:
updatePNList

(((0, 1, -1), (0, 1, -1)),
 ((0, -1, 1), (0, 1, -1)),
 ((-1, 0, 1), (0, 1, -1)),
 ((0, 1, -1), (0, -1, 1)),
 ((-1, 0, 1), (0, -1, 1)),
 ((0, 1, -1), (-1, 0, 1)),
 ((0, -1, 1), (-1, 0, 1)),
 ((-1, 0, 1), (-1, 0, 1)))

# Repressilator

In [187]:
nwStr = "x1:~x2 \n x2:~x3 \n x3:~x1"
network = DSGRN.Network(nwStr)
DSGRN.DrawGraph(network)
parameterGraph = DSGRN.ParameterGraph(network)
parameterGraph.size()

27

In [188]:
notOscillatory = []
size = parameterGraph.size()
for i in range(size):
    parameterSample = parameterGraph.parameter(i)
    domainGraph = DSGRN.DomainGraph(parameterSample)
    morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
    morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)
    l = morseGraph.poset().size()
    notOscillatory.append(parameterSample)
    #print(j)
    for j in range(l):
        if morseGraph.annotation(j)[0].startswith('FC'):
            notOscillatory = notOscillatory[:-1]
            break  
            
len(notOscillatory)

26

In [189]:
pnlist = [stringifyParser(x.stringify()) for x in notOscillatory]
pnlist = [tuple(x) for x in pnlist]
nodeTypeList =  ['celldata/x_o_1_PN.json','celldata/x_o_1_PN.json', 'celldata/x_o_1_PN.json']

In [190]:
updatePNList = compressIt(pnlist, nodeTypeList,float("inf"))

---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2}
buffers:  set()
coord: 0 base length: 3
coord: 1 base length: 3
coord: 2 base length: 3
coordinate:  0
start checking fiber
not redundance neighboring apn:  [(-1, 0, 1), (0, 1, -1)]
start testing the ace is empty or not
start checking fiber
not redundance neighboring apn:  [(0, -1, 1)]
the intersection failed pair:  (-1, 0, 1) (0, -1, 1)
start checking fiber
not redundance neighboring apn:  [(0, -1, 1)]
the intersection failed pair:  (0, 1, -1) (0, -1, 1)
curr:26 updated:26
post states
candidates:  {0, 1, 2}
buffers:  {0}
singles:  set()
---------------------------------------- round ends ----------------------------------------
---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2}
buffers:  {0}
coord: 1 base length: 3
coord: 2 base length: 3
coordinate:  1
start checking fiber
not red

In [191]:
pnlist

[((0, 1, -1), (0, 1, -1), (0, 1, -1)),
 ((0, -1, 1), (0, 1, -1), (0, 1, -1)),
 ((-1, 0, 1), (0, 1, -1), (0, 1, -1)),
 ((0, 1, -1), (0, -1, 1), (0, 1, -1)),
 ((0, -1, 1), (0, -1, 1), (0, 1, -1)),
 ((-1, 0, 1), (0, -1, 1), (0, 1, -1)),
 ((0, 1, -1), (-1, 0, 1), (0, 1, -1)),
 ((0, -1, 1), (-1, 0, 1), (0, 1, -1)),
 ((-1, 0, 1), (-1, 0, 1), (0, 1, -1)),
 ((0, 1, -1), (0, 1, -1), (0, -1, 1)),
 ((0, -1, 1), (0, 1, -1), (0, -1, 1)),
 ((-1, 0, 1), (0, 1, -1), (0, -1, 1)),
 ((0, 1, -1), (0, -1, 1), (0, -1, 1)),
 ((-1, 0, 1), (0, -1, 1), (0, -1, 1)),
 ((0, 1, -1), (-1, 0, 1), (0, -1, 1)),
 ((0, -1, 1), (-1, 0, 1), (0, -1, 1)),
 ((-1, 0, 1), (-1, 0, 1), (0, -1, 1)),
 ((0, 1, -1), (0, 1, -1), (-1, 0, 1)),
 ((0, -1, 1), (0, 1, -1), (-1, 0, 1)),
 ((-1, 0, 1), (0, 1, -1), (-1, 0, 1)),
 ((0, 1, -1), (0, -1, 1), (-1, 0, 1)),
 ((0, -1, 1), (0, -1, 1), (-1, 0, 1)),
 ((-1, 0, 1), (0, -1, 1), (-1, 0, 1)),
 ((0, 1, -1), (-1, 0, 1), (-1, 0, 1)),
 ((0, -1, 1), (-1, 0, 1), (-1, 0, 1)),
 ((-1, 0, 1), (-1, 0, 1),

# p53 Network

In [231]:
with open('osciP53pnlist.json','r') as file:
    pnlist = json.load(file)
    
for i in range(len(pnlist)):
    pnlist[i] = tuple([tuple(x) for x in pnlist[i]])

In [232]:
nodeTypeList = ['celldata/x_o_1_PN.json','celldata/x_o_2_PN.json','celldata/x_o_2_PN.json', \
                'celldata/xy_o_1_PN.json','celldata/(x+y)z_o_2_PN.json']

In [198]:
updatedPNList = compressIt(pnlist, nodeTypeList,4)

---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2, 3, 4}
buffers:  set()
coord: 0 base length: 1
coord: 1 base length: 6
coord: 2 base length: 4
coord: 3 base length: 4
coord: 4 base length: 172
coordinate:  0
start checking fiber
not redundance neighboring apn:  []
start testing the ace is empty or not
curr:6904 updated:6904
post states
candidates:  {0, 1, 2, 3, 4}
buffers:  {0}
singles:  {0}
---------------------------------------- round ends ----------------------------------------
---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2, 3, 4}
buffers:  {0}
coord: 1 base length: 6
coord: 2 base length: 4
coord: 3 base length: 4
coord: 4 base length: 172
coordinate:  2
start checking fiber
not redundance neighboring apn:  [(0, -2, -1, 1)]
start testing the ace is empty or not
is compressed: (-2, 0, -1, 1)
start checking fiber
not redund

In [199]:
len(updatedPNList)

472

In [218]:
# over the full complex not the component anymore but still with attractors

start = time.time()
from homologyHelpers import *
if __name__ == "__main__":
    
    #with open('essentialPNList.dat','r') as file:
    #   pnlist = json.load(file)
    pnlist = tuple([(x[1],x[4]) for x in updatedPNList])
    
    #for i in range(len(pnlist)):
    #    pnlist[i] = [tuple(x) for x in pnlist[i]]

    # pnList for subComplex and preAtt
    subComplexPNList = []
    preAttPNList = []
    
    for pn in pnlist:
        if pn[0] not in [(0, -1, -2, 1),(0, -2, -1, 1)]:
            subComplexPNList.append(pn)
        else:
            preAttPNList.append(pn)

    # face lattice list
    faceLatticeList = []
    for f in ['celldata/x_o_2_FL.json','celldata/(x+y)z_o_2_FL.json']:
        with open(f,'r') as file:
            faceLatticeList.append(json.load(file))

    for i in range(len(faceLatticeList)):
        tempFaceList = dict()
        currFL = faceLatticeList[i]
        for key in currFL:
            tempFaceList[eval(key)] = [tuple(x) for x in currFL[key]]
        faceLatticeList[i] = tempFaceList

    # apnList
    apnList = []
    for f in ['celldata/x_o_2_PN.json','celldata/(x+y)z_o_2_PN.json']:
        apnList.append(loadAtomicChainComplex(f))


    # preAtt
    preAtt = set()
    for i in range(len(preAttPNList)):
    #for i in range(1):
        pn = preAttPNList[i]
        pn = [apnList[j][pn[j]]for j in range(len(apnList))]
        temp = tensorProduct(pn)
        for k in temp:
            preAtt|= temp[k]

    # boundary matrix
    boundaryMatrix = dict()
    coboundaryMatrix = dict()

    # construct boundary matrix of subComplex
    for i in range(len(subComplexPNList)):
    #for i in range(1):
        pn = subComplexPNList[i]
        pn = [apnList[j][pn[j]]for j in range(len(apnList))]
        allFaces = set()
        pnFL = tensorProduct(pn)
        for k in pnFL:
            allFaces|= pnFL[k]

        for face in allFaces:
            if face in boundaryMatrix:
                continue

            boundary = set(product([face[0]], faceLatticeList[1][face[1]]))
            boundary |= set(product(faceLatticeList[0][face[0]],[face[1]]))

            boundaryMatrix[face] = boundary 

    # # construct coboundary matrix of subComplex
    coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

    # construct the subComplex
    subComplex = dict()
    for cell in boundaryMatrix:
        if cell in preAtt:
            continue

        subComplex[cell] = set(boundaryMatrix[cell]) - preAtt

    # discrete morse graph

    queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)
    print('queens: {}, kings: {}, aces: {}'.format(len(queens),len(kings),len(aces)))
    
print(time.time()- start)

queens: 212268, kings: 212268, aces: 0
67.61029505729675


In [224]:
(0, -1, -2, 1),(0, -2, -1, 1)

((0, -1, -2, 1), (0, 1, 2, 3, 4, 5, 6, -1, -2, 7))

In [225]:
d = dict()
for pn in pnlist:
    if pn[0] in [(0, -1, -2, 1),(0, -2, -1, 1)]:
        if pn[0] not in d:
            d[pn[0]] = set()
        d[pn[0]].add(pn[1])

In [226]:
d[(0, -1, -2, 1)] == d[(0, -2, -1, 1)]

True

In [228]:
restPNList = d[(0, -1, -2, 1)] 

In [229]:
apnList = loadAtomicChainComplex('celldata/(x+y)z_o_2_PN.json')
factorAPN = dict()
for pn in restPNList:
    factorAPN[pn] = apnList[pn]

boundaryMatrix = getBoundaryMatrix(factorAPN)
coboundaryMatrix = getCoboundaryMatrix(boundaryMatrix)

preAtt = set()

tempComplex = set()
for i in factorAPN:
    apn = factorAPN[i]
    for d in apn:
        tempComplex|= apn[d]
        
tempComplex-=preAtt

subComplex = dict()
for cell in tempComplex:
    subComplex[cell] = set(boundaryMatrix[cell]) - preAtt
    
queens, kings, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)
print(len(queens),len(kings),len(aces))

15177 15177 1


In [230]:
aces

[(465,)]

In [233]:
updatedPNList = compressIt(pnlist, nodeTypeList,5)

---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2, 3, 4}
buffers:  set()
coord: 0 base length: 1
coord: 1 base length: 6
coord: 2 base length: 4
coord: 3 base length: 4
coord: 4 base length: 172
coordinate:  0
start checking fiber
not redundance neighboring apn:  []
start testing the ace is empty or not
curr:6904 updated:6904
post states
candidates:  {0, 1, 2, 3, 4}
buffers:  {0}
singles:  {0}
---------------------------------------- round ends ----------------------------------------
---------------------------------------- round starts ----------------------------------------
pre states
candidates:  {0, 1, 2, 3, 4}
buffers:  {0}
coord: 1 base length: 6
coord: 2 base length: 4
coord: 3 base length: 4
coord: 4 base length: 172
coordinate:  2
start checking fiber
not redundance neighboring apn:  [(0, -2, -1, 1)]
start testing the ace is empty or not
is compressed: (-2, 0, -1, 1)
start checking fiber
not redund

is compressed: (0, 2, -2, 1, 4, 6, -1, 3, 5, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 4, -2, -1, 2, 3, 5, 6, 7), (0, 1, 4, -2, 2, 5, -1, 3, 6, 7), (0, 1, 4, 5, -2, -1, 2, 3, 6, 7), (0, 1, -2, 4, 5, -1, 2, 3, 6, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 4, -2, 5, -1, 2, 3, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 2, 4, -2, 5, -1, 3, 6, 7), (0, 1, 4, -2, 2, 3, 5, -1, 6, 7), (0, 1, 2, -2, 3, 4, 5, -1, 6, 7), (0, 1, 2, 4, -2, 3, -1, 5, 6, 7), (0, 1, 2, 4, -2, 3, 5, 6, -1, 7), (0, 1, 2, 3, 4, -2, 5, -1, 6, 7), (0, 1, 2, 4, 5, -2, 3, -1, 6, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 2, 4, -2, 3, 5, -1, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 2, 4, -2, 5, -1, 3, 6, 7), (0, 1, 4, -2, 2, 3, 5, 6, -1, 7), (0, 1, 2, -2, 3, 4, 5, 6, -1, 7), (0, 1, 2, 4, -2, 3, 6, -1, 5, 7), (0, 1, 2, 4, 6, -2, 3, 5, -1, 7), (0, 1, 2, 4, -2, 3, -1, 5, 6, 7), (0, 1, 2, 3, 4, -2, 5, 6, -1, 7), (0,

is compressed: (0, 1, 4, 5, -2, 2, 3, 6, -1, 7)
start checking fiber
not redundance neighboring apn:  [(0, -1, 1, 4, -2, 2, 3, 5, 6, 7), (0, 1, -1, 2, 4, 5, -2, 3, 6, 7), (0, 1, -1, 2, 3, 4, 5, -2, 6, 7), (0, 1, 4, -1, 2, -2, 3, 5, 6, 7), (0, -1, 1, 2, -2, 3, 4, 5, 6, 7), (0, 2, -1, 1, 4, -2, 3, 5, 6, 7), (0, 4, -1, 1, 2, -2, 3, 5, 6, 7), (0, 1, 2, -1, 4, -2, 3, 5, 6, 7)]
start testing the ace is empty or not
is compressed: (0, -1, 1, 2, 4, -2, 3, 5, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 4, -2, 2, 3, 5, -1, 6, 7), (0, 1, 4, -2, 2, -1, 3, 5, 6, 7), (0, 1, 4, -2, 2, 5, 6, -1, 3, 7), (0, 1, 2, 4, 5, -2, -1, 3, 6, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 4, 5, -2, 2, -1, 3, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, -1, 4, -2, 2, 3, 5, 6, 7), (0, 2, -1, 1, 4, -2, 3, 5, 6, 7), (0, 4, -1, 1, -2, 2, 3, 5, 6, 7), (0, -1, 1, -2, 2, 3, 4, 5, 6, 7)]
start testing the ace is empty or not
is compressed: (0, -1, 1, 4, -2, 

is compressed: (0, -2, 2, 4, -1, 1, 3, 5, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 4, -2, 2, -1, 1, 3, 5, 6, 7), (0, 1, 4, -2, -1, 2, 3, 5, 6, 7), (0, 1, 2, 4, -2, -1, 3, 5, 6, 7), (0, 1, -2, 2, 3, 4, -1, 5, 6, 7), (0, 1, 4, -2, 2, 6, -1, 3, 5, 7), (0, 1, -2, 2, -1, 3, 4, 5, 6, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 4, -2, 2, -1, 3, 5, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 2, 4, 5, 6, -1, 3, -2, 7), (0, 1, 2, 4, 6, -1, 5, -2, 3, 7), (0, 1, 2, 4, 6, -2, 5, -1, 3, 7), (0, 1, 2, 4, 5, -2, 6, -1, 3, 7), (0, 1, 2, 3, 4, 5, 6, -2, -1, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 2, 4, 5, 6, -2, -1, 3, 7)
start checking fiber
not redundance neighboring apn:  [(0, 2, -2, 4, 6, -1, 1, 3, 5, 7), (0, 4, -2, 2, 6, -1, 1, 3, 5, 7)]
start testing the ace is empty or not
is compressed: (0, -2, 2, 4, 6, -1, 1, 3, 5, 7)
start checking fiber
not redundance neighboring apn:  [(0, 2, 4, -2, 6, -1, 1, 3, 5, 7)

is compressed: (0, 1, 2, 3, 4, -2, 5, 6, -1, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 2, 3, -2, 4, -1, 5, 6, 7), (0, 1, 2, -2, 3, 4, 6, -1, 5, 7), (0, 1, 2, 4, -2, 3, -1, 5, 6, 7), (0, 1, -2, 2, 3, 4, -1, 5, 6, 7), (0, 1, 2, -2, 4, -1, 3, 5, 6, 7), (0, 1, 2, -2, -1, 3, 4, 5, 6, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 2, -2, 3, 4, -1, 5, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 2, 3, -1, 4, 5, -2, 6, 7), (0, 1, 2, 3, 4, 6, -1, 5, -2, 7), (0, 1, 2, 4, -1, 3, 5, -2, 6, 7), (0, 1, 2, 3, 4, -1, -2, 5, 6, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 2, 3, 4, -1, 5, -2, 6, 7)
start checking fiber
not redundance neighboring apn:  [(0, 1, 2, 3, -1, 4, 5, -2, 6, 7), (0, 1, 2, 4, 5, -1, -2, 3, 6, 7), (0, 1, 2, 4, 5, 6, -1, 3, -2, 7), (0, 1, 2, 4, 6, -1, 3, 5, -2, 7), (0, 1, 2, 4, -1, 3, -2, 5, 6, 7), (0, 1, 2, 3, 4, 6, -1, 5, -2, 7)]
start testing the ace is empty or not
is compressed: (0, 1, 2, 4, -1, 3

KeyboardInterrupt: 

In [238]:
l = {(-1, 0, 1): {0: [[2], [1], [4]], 1: [[1, 2], [2, 4], [1, 4]], 2: [[1, 2, 4]]}}

In [251]:
subComplex = getBoundaryMatrix(l)
for k in subComplex:
    subComplex[k] = set(subComplex[k])
coboundaryMatrix = getCoboundaryMatrix(subComplex)
preAtt = set()

In [253]:
kings, queens, aces = genMatchingWithPreAtt(subComplex,coboundaryMatrix,preAtt)

In [246]:
subComplex

{(1, 2, 4): [(1, 2), (2, 4), (1, 4)],
 (1, 2): [(2,), (1,)],
 (2, 4): [(2,), (4,)],
 (1, 4): [(1,), (4,)],
 (2,): [],
 (1,): [],
 (4,): []}

In [249]:
coboundaryMatrix

{(1, 2): [(1, 2, 4)],
 (2, 4): [(1, 2, 4)],
 (1, 4): [(1, 2, 4)],
 (2,): [(1, 2), (2, 4)],
 (1,): [(1, 2), (1, 4)],
 (4,): [(2, 4), (1, 4)]}